In [1]:
import pandas as pd
import numpy as np

# Опыт с MultiLabelBinarizer

**Создадим набор данных**

In [312]:
d = pd.DataFrame({'text': ['Сладкий чай', 'Милое лицо', 'Сладкий чай'], 
              't': ['Чай', 'Лицо', 'Чай'], 
              'p': ['Чай', 'Лицо', 'Ягода']})
d.head()

,text,t,p
0,Сладкий чай,Чай,Чай
1,Милое лицо,Лицо,Лицо
2,Сладкий чай,Чай,Ягода


In [292]:
agr = d.groupby(by='text', group_keys=False).agg(t=('t','unique'), p=('p','unique'))
agr.head()

,t,p
text,,
Милое лицо,[Лицо],[Лицо]
Сладкий чай,[Чай],"[Чай, Ягода]"


In [284]:
agr = d.groupby(by='text').agg(['unique'])
agr.columns = agr.columns.droplevel(1)
agr.head()

,t,p
text,,
Милое лицо,[Лицо],[Лицо]
Сладкий чай,[Чай],"[Чай, Ягода]"


In [285]:
d[['t', 'p']].agg('unique')

array(['Лицо', 'Чай', 'Ягода'], dtype=object)

In [286]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import precision_score, recall_score

mlb = MultiLabelBinarizer(classes=d[['t', 'p']].agg('unique'))

gold_labels = mlb.fit_transform(agr['t'])
pred_labels = mlb.transform(agr['p'])

recall_score(gold_labels, pred_labels, average='samples')

1.0

In [69]:
train = pd.read_csv('data/processed/perfumery_train.csv').sort_values('frequency', ascending=False)[['phrase', 'subtopic']]
train.head()

,phrase,subtopic,frequency,lemmas
0,флакон,Флакон,88474,флакон
1,шлейф,След аромата,51317,шлейф
2,роза,Цветочные,35609,роза
3,раскрывается,Время раскрытия,33018,раскрываться
4,нота,Нота,31010,нота


In [261]:
all_metrics = pd.read_csv('data/model/0.85_500_all_metrics.csv')
all_metrics.tail()

,f1,precision,recall,validation_size,model_from_val,model_from_all,people_from_val
38,0.926896,0.905446,0.986315,19657,19235,0,19500
39,0.925339,0.903805,0.985811,20157,19778,0,20000
40,0.925578,0.903919,0.985380,20657,20275,0,20500
41,0.924749,0.902891,0.984970,21157,20727,0,21000
42,0.927303,0.905535,0.988445,21550,21226,0,21393


In [262]:
m = all_metrics[-3:][['f1', 'precision', 'recall']].agg('mean')
m

f1           0.925877
precision    0.904115
recall       0.986265
dtype: float64

In [263]:
all_metrics.iloc[-1:, :3]

,f1,precision,recall
42,0.927303,0.905535,0.988445


In [272]:
all_metrics.iloc[-1:, :3] = all_metrics.iloc[-3:, :3].agg('mean')

In [267]:
all_metrics[-3:][['f1', 'precision', 'recall']].agg('mean')

f1           0.925402
precision    0.903642
recall       0.985538
dtype: float64

In [293]:
x = [[1], [2], [1, 2]]

In [298]:
np.array(x, dtype='object')[[1, 2]]

array([list([2]), list([1, 2])], dtype=object)

### Проверка двойного explode 

In [313]:
d = pd.DataFrame({'text': ['11'], 
              't': [['Громкость', 'Цветочные', 'Горький']], 
              'p': [['Цвет', 'Гром']]})
d.head()
g = d.explode('t').explode('p')
g

,text,t,p
0,11,Громкость,Цвет
0,11,Громкость,Гром
0,11,Цветочные,Цвет
0,11,Цветочные,Гром
0,11,Горький,Цвет
0,11,Горький,Гром


In [315]:
g.groupby(by='text').agg(t=('t', 'unique'), p=('p', 'unique'))

,t,p
text,,
11,"[Громкость, Цветочные, Горький]","[Цвет, Гром]"


## Отладка ошибки по Explode

In [128]:
test = pd.read_csv('test.csv')
test.head()

,phrase,subtopic,true
0,аромат альдегидов,Альдегида,Альдегида
1,аромат альдегидов,Аромат листьев,Альдегида
2,аромат альдегидов,Аромат смолы,Альдегида
3,аромат альдегидов,Аромат табака,Альдегида
4,аромат альдегидов,Аромат дыма,Альдегида


**Загрузка данных**

In [129]:
t = test.groupby(by='phrase').agg(subtopic=('subtopic', 'unique'), true=('true', 'unique'))[:5]
t

,subtopic,true
phrase,,
аромат альдегидов,"[Альдегида, Аромат листьев, Аромат смолы, Аром...",[Альдегида]
аромат ананаса,"[Тропический аромат, Алкоголя/спирта, Аромат л...",[Тропический аромат]
аромат ангелов,"[Название бренда/автора, Алкоголя/спирта, Альд...",[Название аромата]
аромат апельсина,"[Цитрусовые, Тропический аромат, Аромат листье...",[Цитрусовые]
аромат апельсинового цвета,"[Цветочные, Цитрусовые, Шампанского, Аромат ли...",[Цвет]


In [130]:
labels = np.array([], dtype='object')
for i in range(len(t)):
    labels = np.append(labels, [*t['subtopic'][i], *t['true'][i]])
labels = np.unique(labels)

In [122]:
t['subtopic']

phrase
аромат альдегидов             [Альдегида, Аромат листьев, Аромат смолы, Аром...
аромат ананаса                [Тропический аромат, Алкоголя/спирта, Аромат л...
аромат ангелов                [Название бренда/автора, Алкоголя/спирта, Альд...
аромат апельсина              [Цитрусовые, Тропический аромат, Аромат листье...
аромат апельсинового цвета    [Цветочные, Цитрусовые, Шампанского, Аромат ли...
Name: subtopic, dtype: object

In [123]:
list(t['subtopic'][0])

['Альдегида', 'Аромат листьев', 'Аромат смолы', 'Аромат табака', 'Аромат дыма']

In [139]:
t['subtopic'][0] = np.append(t['subtopic'][0], ['Тропический аромат'])
t

,subtopic,true
phrase,,
аромат альдегидов,"[Альдегида, Аромат листьев, Аромат смолы, Аром...","[Альдегида, Тропический аромат]"
аромат ананаса,"[Тропический аромат, Алкоголя/спирта, Аромат л...",[Тропический аромат]
аромат ангелов,"[Название бренда/автора, Алкоголя/спирта, Альд...",[Название аромата]
аромат апельсина,"[Цитрусовые, Тропический аромат, Аромат листье...",[Цитрусовые]
аромат апельсинового цвета,"[Цветочные, Цитрусовые, Шампанского, Аромат ли...",[Цвет]


In [155]:
t['true'][4] = np.append(t['true'][4], ['Цитрусовые'])
t

,subtopic,true
phrase,,
аромат альдегидов,"[Альдегида, Аромат листьев, Аромат смолы, Аром...","[Альдегида, Тропический аромат, Аромат листьев]"
аромат ананаса,"[Тропический аромат, Алкоголя/спирта, Аромат л...",[Тропический аромат]
аромат ангелов,"[Название бренда/автора, Алкоголя/спирта, Альд...",[Название аромата]
аромат апельсина,"[Цитрусовые, Тропический аромат, Аромат листье...",[Цитрусовые]
аромат апельсинового цвета,"[Цветочные, Цитрусовые, Шампанского, Аромат ли...","[Цвет, Цветочные, Шампанского, Цитрусовые]"


In [167]:
t

,subtopic,true
phrase,,
аромат альдегидов,"[Альдегида, Аромат листьев, Аромат смолы, Аром...","[Альдегида, Тропический аромат, Аромат листьев]"
аромат ананаса,"[Тропический аромат, Алкоголя/спирта, Аромат л...",[Тропический аромат]
аромат ангелов,"[Название бренда/автора, Алкоголя/спирта, Альд...",[Название аромата]
аромат апельсина,"[Цитрусовые, Тропический аромат, Аромат листье...",[Цитрусовые]
аромат апельсинового цвета,"[Цветочные, Цитрусовые, Шампанского, Аромат ли...","[Цвет, Цветочные, Шампанского, Цитрусовые]"


In [168]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import precision_score, recall_score, f1_score

mlb = MultiLabelBinarizer(classes=labels)

gold_labels = mlb.fit_transform(t['true'].values)
pred_labels = mlb.transform(t['subtopic'].values)

precision_score(y_pred=pred_labels, y_true=gold_labels, average='samples')

0.3

In [162]:
recall_score(y_pred=pred_labels, y_true=gold_labels, average='samples')

0.75

In [163]:
f1_score(pred_labels, gold_labels, average='samples')

0.4